In [2]:
import requests as r
import csv
from bs4 import BeautifulSoup
import re 

In [3]:
# file path
fp = 'C:\\Users\\Luis\\MS-DS\\Projects\\'
filename = 'hotels.csv'

In [4]:
#how many pages to search

site = 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX'

In [5]:
result = r.get(site)

In [6]:
soup = BeautifulSoup(result.content, 'lxml')

In [ ]:
with open(fp + filename, 'w', newline='') as dataout:
  datawriter = csv.writer(dataout, delimiter =',', quotechar='"', quoting= csv.QUOTE_NONNUMERIC)
  datawriter.writerow(['hotel_name', 'address', 'phone_number'])

In [7]:
r.get('https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C%20TX&page=1')

<Response [200]>

In [8]:
# This works
links = []
i=0
for i in range(8):
  links.append(site + '&page=' + str(i+1))




In [10]:
# this works 
hotel_name=[]
phone = []
address=[]
for link in links:
  result = r.get(link)
  soup = BeautifulSoup(result.content,'lxml')
  soup.find_all('a', attrs = {'class': 'business-name'})
  for hotel in soup.find_all('h2'):
    hotel_name.append(hotel.text)
  for num in soup.find_all('div', attrs={'class': 'phones phone primary'}):
    phone.append(num.text)
  for addy in soup.find_all('div', attrs={'class': 'street-address'}):
    address.append(addy.text)

In [29]:
links

['https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=1',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=2',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=3',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=4',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=5',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=6',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=7',
 'https://www.yellowpages.com/search?search_terms=hotel&geo_location_terms=Austin%2C+TX&page=8']

In [13]:
s = [1,2,3,4,5,6,7,8,9,0]
starts = [str(x)for x in s]

In [33]:
for hotel in hotel_name:
  if hotel[0] not in starts:
    hotel_name.remove(hotel)

In [38]:
len(phone)

240